# Data Augmentation ( YOLO Format )

Inspired by https://medium.com/red-buffer/apply-data-augmentation-on-yolov5-yolov8-dataset-958e89d4bc5d

In the YOLO format, bounding box coordinates are typically represented as (class, x_center, y_center, width, height). Here's what each component means:

- **class:** The class of the annotated object.
- **x_center:** The x-coordinate of the center of the bounding box relative to the width of the image.
- **y_center:** The y-coordinate of the center of the bounding box relative to the height of the image.
- **width:** The width of the bounding box relative to the total width of the image.
- **height:** The height of the bounding box relative to the total height of the image.

If you have labels in text format created from LabelMeToYOLO, please use the "convert_to_yolo" function below to transform the labels.

Instalation of packages (Run this cell only once)


In [ ]:
# ! pip install albumentations

In [ ]:
# ! pip install -r requirements.txt

### Useful Functions Definition

In [1]:
# définition des fonctions utiles

import albumentations as A
import cv2
import os
import yaml
import pybboxes as pbx


with open("augmentation.yaml", 'r') as stream:
    CONSTANTS = yaml.safe_load(stream)


def is_image_by_extension(file_name):
    """
    Check if the given file has a recognized image extension.

    Args:
        file_name (str): Name of the file.

    Returns:
        bool: True if the file has a recognized image extension, False otherwise.

    """
    # List of common image extensions
    image_extensions = ['jpg', 'jpeg', 'png', 'gif', 'bmp', 'tiff', 'webp']
    # Get the file extension
    file_extension = file_name.lower().split('.')[-1]
    # Check if the file has a recognized image extension
    return file_extension in image_extensions


def get_inp_data(img_file):
    """
    Get input data for image processing.

    Args:
        img_file (str): Name of the input image file.

    Returns:
        tuple: A tuple containing the image, ground truth bounding boxes, and augmented file name.

    """
    file_name = os.path.splitext(img_file)[0]
    aug_file_name = f"{file_name}_{CONSTANTS['transformed_file_name']}"
    image = cv2.imread(os.path.join(CONSTANTS["inp_img_pth"], img_file))
    lab_pth = os.path.join(CONSTANTS["inp_lab_pth"], f"{file_name}.txt")
    gt_bboxes = get_bboxes_list(lab_pth, CONSTANTS['CLASSES'])
    return image, gt_bboxes, aug_file_name


def get_album_bb_list(yolo_bbox, class_names):
    """
    Extracts bounding box information for a single object from YOLO format.

    Args:
        yolo_bbox (str): YOLO format string representing bounding box information.
        class_names (list): List of class names corresponding to class numbers.

    Returns:
        list: A list containing [x_center, y_center, width, height, class_name].
    """
    str_bbox_list = yolo_bbox.split()
    class_number = int(str_bbox_list[0])
    class_name = class_names[class_number]
    bbox_values = list(map(float, str_bbox_list[1:]))
    album_bb = bbox_values + [class_name]
    return album_bb


def get_album_bb_lists(yolo_str_labels, classes):
    """
    Extracts bounding box information for multiple objects from YOLO format.

    Args:
        yolo_str_labels (str): YOLO format string containing bounding box information for multiple objects.
        classes (list): List of class names corresponding to class numbers.

    Returns:
        list: A list of lists, each containing [x_center, y_center, width, height, class_name].
    """
    album_bb_lists = []
    yolo_list_labels = yolo_str_labels.split('\n')
    for yolo_str_label in yolo_list_labels:
        if yolo_str_label:
            album_bb_list = get_album_bb_list(yolo_str_label, classes)
            album_bb_lists.append(album_bb_list)
    return album_bb_lists


def get_bboxes_list(inp_lab_pth, classes):
    """
    Reads YOLO format labels from a file and returns bounding box information.

    Args:
        inp_lab_pth (str): Path to the YOLO format labels file.
        classes (list): List of class names corresponding to class numbers.

    Returns:
        list: A list of lists, each containing [x_center, y_center, width, height, class_name].
    """
    yolo_str_labels = open(inp_lab_pth, "r").read()

    if not yolo_str_labels:
        print("No object")
        return []

    lines = [line.strip() for line in yolo_str_labels.split("\n") if line.strip()]
    album_bb_lists = get_album_bb_lists("\n".join(lines), classes) if len(lines) > 1 else [get_album_bb_list("\n".join(lines), classes)]

    return album_bb_lists


def single_obj_bb_yolo_conversion(transformed_bboxes, class_names):
    """
    Convert bounding boxes for a single object to YOLO format.

    Parameters:
    - transformed_bboxes (list): Bounding box coordinates and class name.
    - class_names (list): List of class names.

    Returns:
    - list: Bounding box coordinates in YOLO format.
    """
    if transformed_bboxes:
        class_num = class_names.index(transformed_bboxes[-1])
        bboxes = list(transformed_bboxes)[:-1]
        bboxes.insert(0, class_num)
    else:
        bboxes = []
    return bboxes


def multi_obj_bb_yolo_conversion(aug_labs, class_names):
    """
    Convert bounding boxes for multiple objects to YOLO format.

    Parameters:
    - aug_labs (list): List of bounding box coordinates and class names.
    - class_names (list): List of class names.

    Returns:
    - list: List of bounding box coordinates in YOLO format for each object.
    """
    yolo_labels = [single_obj_bb_yolo_conversion(aug_lab, class_names) for aug_lab in aug_labs]
    return yolo_labels


def save_aug_lab(transformed_bboxes, lab_pth, lab_name):
    """
    Save augmented bounding boxes to a label file.

    Args:
        transformed_bboxes (list): List of augmented bounding boxes.
        lab_pth (str): Path to the output label directory.
        lab_name (str): Name of the label file.

    """
    lab_out_pth = os.path.join(lab_pth, lab_name)
    with open(lab_out_pth, 'w') as output:
        for bbox in transformed_bboxes:
            updated_bbox = str(bbox).replace(',', ' ').replace('[', '').replace(']', '')
            output.write(updated_bbox + '\n')


def save_aug_image(transformed_image, out_img_pth, img_name):
    """
    Save augmented image to an output directory.

    Args:
        transformed_image (numpy.ndarray): Augmented image.
        out_img_pth (str): Path to the output image directory.
        img_name (str): Name of the image file.

    """
    out_img_path = os.path.join(out_img_pth, img_name)
    cv2.imwrite(out_img_path, transformed_image)


def draw_yolo(image, labels, file_name):
    """
    Draw bounding boxes on an image based on YOLO format.

    Args:
        image (numpy.ndarray): Input image.
        labels (list): List of bounding boxes in YOLO format.

    """
    H, W = image.shape[:2]
    for label in labels:
        yolo_normalized = label[1:]
        box_voc = pbx.convert_bbox(tuple(yolo_normalized), from_type="yolo", to_type="voc", image_size=(W, H))
        cv2.rectangle(image, (box_voc[0], box_voc[1]),
                      (box_voc[2], box_voc[3]), (0, 0, 255), 1)
    cv2.imwrite(f"bb_image/{file_name}.jpg", image)
    # cv2.imshow(f"{file_name}.png", image)
    # cv2.waitKey(0)


def has_negative_element(lst):
    """
    Check if the given list contains any negative element.

    Args:
        lst (list): List of elements.

    Returns:
        bool: True if there is any negative element, False otherwise.
    """
    return None
    # return any(x < 0 for x in lst)


def get_augmented_results(image, bboxes):
    """
    Apply data augmentation to an input image and bounding boxes.

    Parameters:
    - image (numpy.ndarray): Input image.
    - bboxes (list): List of bounding boxes in YOLO format [x_center, y_center, width, height, class_name].

    Returns:
    - tuple: A tuple containing the augmented image and the transformed bounding boxes.
    """
    # Define the augmentations
    transform = A.Compose([
        A.Rotate(limit=25, p=0.2),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0,p=0.4),
    ], bbox_params=A.BboxParams(format='yolo'))

    # Apply the augmentations
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image, transformed_bboxes = transformed['image'], transformed['bboxes']
    
    return transformed_image, transformed_bboxes


def has_negative_element(matrix):
    """
    Check if there is a negative element in the 2D list of augmented bounding boxes.

    Args:
        matrix (list[list]): The 2D list.

    Returns:
        bool: True if a negative element is found, False otherwise.

    """
    return None
    # return any(element < 0 for row in matrix for element in row)


def save_augmentation(trans_image, trans_bboxes, trans_file_name):
    """
    Saves the augmented label and image if no negative elements are found in the transformed bounding boxes.

    Parameters:
        trans_image (numpy.ndarray): The augmented image.
        trans_bboxes (list): The transformed bounding boxes.
        trans_file_name (str): The name for the augmented output.

    Returns:
        None
    """
    tot_objs = len(trans_bboxes)
    if tot_objs:
        # Convert bounding boxes to YOLO format
        trans_bboxes = multi_obj_bb_yolo_conversion(trans_bboxes, CONSTANTS['CLASSES']) if tot_objs > 1 else [single_obj_bb_yolo_conversion(trans_bboxes[0], CONSTANTS['CLASSES'])]
        if not has_negative_element(trans_bboxes):
            # Save augmented label and image
            save_aug_lab(trans_bboxes, CONSTANTS["out_lab_pth"], trans_file_name + ".txt")
            save_aug_image(trans_image, CONSTANTS["out_img_pth"], trans_file_name + ".jpg")
            # Draw bounding boxes on the augmented image
            draw_yolo(trans_image, trans_bboxes, trans_file_name)
        else:
            print("Found Negative element in Transformed Bounding Box...")
    else:
        print("Label file is empty")

def run_yolo_augmentor():
    """
    Run the YOLO augmentor on a set of images.

    This function processes each image in the input directory, applies augmentations,
    and saves the augmented images and labels to the output directories.

    """
    imgs = [img for img in os.listdir(CONSTANTS["inp_img_pth"]) if is_image_by_extension(img)]

    for img_num, img_file in enumerate(imgs):
        print(f"{img_num+1}-image is processing...\n")
        image, gt_bboxes, aug_file_name = get_inp_data(img_file)
        aug_img, aug_label = get_augmented_results(image, gt_bboxes)
        if len(aug_img) and len(aug_label):
            save_augmentation(aug_img, aug_label, aug_file_name)

$$ \textbf{Attention : You may need to modify the augmentation.yaml file to specify the paths.}


### Yaml file (augmentation.yaml)
- **inp_img_pth:** path to the input images directory
- **inp_lab_pth:** path to the input labels directory
- **out_img_pth:** path to the output images directory
- **out_lab_pth:** path to the output labels directory
- **transformed_file_name:** `aug_out`
- **CLASSES:** `["undamagedresidentialbuilding", "undamagedcommercialbuilding", "damagedresidentialbuilding", "damagedcommercialbuilding"]`


Running the data augmentation on the input folders and saving the results in the output folders defined in the augmentation.yaml file.

In [3]:
run_yolo_augmentor()

1-image is processing...

2-image is processing...

3-image is processing...

4-image is processing...

5-image is processing...

6-image is processing...

7-image is processing...

8-image is processing...

9-image is processing...

10-image is processing...

11-image is processing...

12-image is processing...

13-image is processing...

14-image is processing...

15-image is processing...

16-image is processing...

17-image is processing...

18-image is processing...

19-image is processing...

20-image is processing...

21-image is processing...

22-image is processing...

23-image is processing...

24-image is processing...

25-image is processing...

26-image is processing...

27-image is processing...

28-image is processing...

29-image is processing...

30-image is processing...

31-image is processing...

32-image is processing...

33-image is processing...

34-image is processing...

35-image is processing...

36-image is processing...

37-image is processing...

38-image i

# Label conversion 

Change the source and output folders

In [1]:
import os

# Function to convert annotations to YOLO format
def convert_to_yolo(annotation_file, output_file):
    with open(annotation_file, 'r') as f:
        lines = f.readlines()
    
    yolo_annotations = []
    for line in lines:
        line = line.strip().split()
        # Get normalized coordinates
        x1, y1, x2, y2, x3, y3, x4, y4 = map(float, line[1:])
        
        # Calculate center coordinates, width, and height
        width = x2 - x1
        height = y3 - y2
        x_center = x1 + width / 2
        y_center = y2 + height / 2

        # Convert coordinates to YOLO format
        class_label = int(line[0])  # object class
        yolo_format = f"{class_label} {x_center} {y_center} {width} {height}\n"
        yolo_annotations.append(yolo_format)

    # Write converted annotations to an output file
    with open(output_file, 'w') as f:
        for annotation in yolo_annotations:
            f.write(annotation)

# Directory containing annotations to convert
input_directory = ""
# Output directory for converted annotations
output_directory = ""

# Iterate through annotation files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".txt"):
        annotation_file = os.path.join(input_directory, filename)
        output_file = os.path.join(output_directory, filename)

        # Convert annotation to YOLO format
        convert_to_yolo(annotation_file, output_file)


In the case where polygons are used as labels, it works for any type of annotations, even mixed ones (polygon + rectangle).

In [4]:
import os

# Function to calculate the bounding rectangle of a set of points
def calculate_bounding_rectangle(x_coords, y_coords):
    min_x, min_y = min(x_coords), min(y_coords)
    max_x, max_y = max(x_coords), max(y_coords)
    width = max_x - min_x
    height = max_y - min_y
    x_center = min_x + width / 2
    y_center = min_y + height / 2
    return x_center, y_center, width, height

# Function to convert polygon annotations to YOLO format with bounding rectangles
def convert_to_yolo2(annotation_file, output_file):
    with open(annotation_file, 'r') as f:
        lines = f.readlines()

    yolo_annotations = []
    for line in lines:
        line = line.strip().split()
        
        # Retrieve polygon coordinates
        class_label = int(line[0])  # object class
        num_points = (len(line) - 1) // 2
        x_coords = [float(line[i]) for i in range(1, num_points * 2, 2)]
        y_coords = [float(line[i]) for i in range(2, num_points * 2, 2)]
        
        # Calculate the bounding rectangle
        x_center, y_center, width, height = calculate_bounding_rectangle(x_coords, y_coords)

        # Convert coordinates to YOLO format
        yolo_format = f"{class_label} {x_center} {y_center} {width} {height}\n"
        yolo_annotations.append(yolo_format)

    # Write the converted annotations to an output file
    with open(output_file, 'w') as f:
        for annotation in yolo_annotations:
            f.write(annotation)

# Directory containing the annotations to convert
# input_directory = ""
# Output directory for the converted annotations
# output_directory = ""

# Traverse annotation files in the input directory
# for filename in os.listdir(input_directory):
#     if filename.endswith(".txt"):
#         annotation_file = os.path.join(input_directory, filename)
#         output_file = os.path.join(output_directory, filename)

#         # Convert the annotation to YOLO format
#         convert_to_yolo(annotation_file, output_file)
